# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. **This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").**

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [21]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [27]:
def date_sorter():
    
    order = None
    # YOUR CODE HERE
    import re
    try:
        # Read file into a Pandas Series
        with open('assets/dates.txt', 'r') as file:
            df = pd.Series(file.readlines())  # Directly read all lines
        ##print(df.columns)
        ##df_dates = df.copy()
        
        # Convert to DataFrame
        df_dates = pd.DataFrame(df.copy(), columns=["text"])
        print(df_dates)
        
        # Define date patterns (regex)
        date_patterns = [
            r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,]?\s*\d{4})',
            r'(\d{1,2}/\d{1,2}/\d{2,4})',
            r'\b(?!\d{3}-\d{3}-\d{4}\b)(\d{1,2}-\d{1,2}-\d{2,4})',
            r'(?<!\d)(?:[a-zA-Z])?(\d{1,2}[-/]\d{1,2}[-/]\d{2,4})(?!-\d{4})',
            r'(?<!\d)(?:[a-zA-Z])?(\d{1,2}/\d{2,4})(?!-\d{4})',
            r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,]?\s*\d{0,2},?\s*\d{4})',
            r'(\d{1,2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{4})',
            r'(\d{1,2}/\d{4})',
            r'((?:19\d{2}|20[0-2]\d|202[0-4]))'
        ]

        month_corrections = {
        "Janaury": "January",
        "Decemeber": "December",   # Example of a partial misspelling
        } 
        
        # Combine all patterns into one
        combined_pattern = "|".join(date_patterns)

        # Extract dates and the line they were found in
        
        df_dates['extracted_date'] = df_dates['text'].apply(
            lambda text: next(
            (group for m in re.finditer(combined_pattern, text) for group in m.groups() if group),
            None
            )
        )
        
        df_dates['parsed_date'] = df_dates['extracted_date'].apply(
            lambda x: ' '.join([month_corrections.get(word, word) for word in x.split()]) if pd.notnull(x) else x
        )
        
        
        pattern = re.compile(
            r'\b(?:\d{1,2}[-/]\d{1,2}[-/](\d{2,4}))\b'            # Group 1: MM/DD/YY or MM/DD/YYYY
            r'|\b(?:\d{1,2}[-/](\d{2,4}))\b'                      # Group 2: MM/YYYY or MM/YY
            ##r'\b(?:[^\d]?(\d{1,2})[-/](\d{2,4}))\b' 
            # Group 2: MM/YYYY or MM/YY
            r'|\b(19\d{2}|20[0-2][0-9]|202[0-4])\b'               # Group 3: just a year
            r'|\b(\d{1,2}) ((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*) (\d{4})\b'
            # Groups:# 4 = day (e.g., "12") # 5 = month name (e.g., "Feb") # 6 = year (e.g., "2002")
            r'|\b((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,]?)\s+(\d{0,2})[.,]?\s*(\d{4})\b'  
            # Groups 7-9: Jan 1, 2021
        )

        df_dates['year'] = df_dates['parsed_date'].apply(
        lambda x: 
        (
            (
                m := pattern.search(str(x)))
                and 
                    (
                        (
                            #y := next((g for g in m.groups()[::1] if g and g.isdigit()), None))
                            y := next((g for g in reversed(m.groups()) if g and re.fullmatch(r'\d{2,4}', g)), None))
                and 
                    (
                        '19' + y if len(y) == 2 and int(y) > 24 else
                            '20' + y if len(y) == 2 else
                                y
                    )
                )
            ) 
            if pattern.search(str(x)) else ''
        )
        
        # Extract month inline
        df_dates['month'] = df_dates['parsed_date'].apply(
        lambda x: 
        (
            (
                m := pattern.search(str(x))) and (
                (
                    (lambda p: p[0] if p and p[0].isdigit() else '')(
                        re.split(r'[-/]', str(x)) if re.match(r'^\d{1,2}[-/]', str(x)) else []
                    )
                )
            or (m.group(5) if m.group(5) else '')
            or (m.group(7) if m.group(7) else '')
            )
            ) or '01'
        )
        
        # Extract the day 
        df_dates['day'] = df_dates['parsed_date'].apply(
        lambda x: (
        (m := pattern.search(str(x))) and (
            # Only if date has 3 parts (e.g., MM/DD/YYYY or DD-MM-YYYY)
            (lambda p: p[1].zfill(2) if len(p) == 3 and p[1] and p[1].isdigit() else '')(
                re.split(r'[-/]', str(x)) if re.match(r'^\d{1,2}[-/]\d{1,2}[-/]\d{2,4}$', str(x).zfill(2)) else []
            )
            # From "12 Feb 2002"
            or (m.group(4).zfill(2) if m.lastindex and m.lastindex >= 4 and m.group(4) and m.group(4).isdigit() else '')
            # From "Feb 12, 2002"
            or (m.group(8).zfill(2) if m.lastindex and m.lastindex >= 8 and m.group(8) and m.group(8).isdigit() else '')
        )
            ) or '01'
        )

        
        df_dates['month'] = df_dates['month'].apply(
        lambda m: (
        str(m).zfill(2) if str(m).isdigit() and 1 <= int(m) <= 12 else (
            (match := re.search(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*', str(m), re.IGNORECASE))
            and {
                'jan': '01', 'feb': '02', 'mar': '03', 'apr': '04', 'may': '05',
                'jun': '06', 'jul': '07', 'aug': '08', 'sep': '09',
                'oct': '10', 'nov': '11', 'dec': '12'
            }.get(match.group(1).lower(), '')
        )
            ) or ''
        )        
        
        df_dates['normalized_date'] = df_dates['year'] + '-' + df_dates['month'] + '-' + df_dates['day']
    
        df_dates['original_index']= df_dates.index 
        
        sorted_df_dates = df_dates.sort_values(by=['normalized_date', 'original_index']).reset_index(drop=True)
        
        sorted_df_dates['sorted_index']= range(len(df_dates))
        
        
        print(sorted_df_dates['original_index'])
        #df_dates.to_csv('output.csv', index=True, encoding='utf-8', quoting=1)
        sorted_df_dates.to_csv('output.csv', index=True, encoding='utf-8', quoting=1)

        ##print(sorted_df_dates['original_index'].apply(type))
        return sorted_df_dates['original_index']
    except FileNotFoundError:
                print("Error: File 'assets/dates.txt' not found.")
    return None  # Return None if file is missing
    raise NotImplementedError()
    ##return order # Your answer here
s_test = date_sorter()

                                                  text
0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
..                                                 ...
495  1979 Family Psych History: Family History of S...
496  therapist and friend died in ~2006 Parental/Ca...
497                       2008 partial thyroidectomy\n
498  sPt describes a history of sexual abuse as a c...
499  . In 1980, patient was living in Naples and de...

[500 rows x 1 columns]
0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Name: original_index, Length: 500, dtype: int64


In [28]:
def run_df_modified_check():
    
    #Check if df appears to be modified.
    
    print(type(df) == pd.Series)
    print(df.index == pd.RangeIndex(start=0, stop=500, step=1))
    print(df.apply(type))
    print(df.apply(type) == str)
    #print(s_test.apply(type) == str)
    print(df.astype(str).str.len().min() >= 6)
    
    #print(s_test.astype(str).str[5].dropna().apply(ord).sum() == 38354)

    
    try:
        assert type(df) == pd.Series
        assert (df.index == pd.RangeIndex(start=0, stop=500, step=1)).all()
        assert (df.apply(type) == str).all()
        assert df.str.len().min() >= 6
        assert df.str[5].apply(ord).sum() == 38354
        print("Passed df modification check")
    except:
        print("Failed df modification check")

run_df_modified_check()

# check if running the code twice produces the same result
try:
    assert (date_sorter() == s_test).all()
    print("Passed repeatability check")
except:
   print("Failed repeatability check")

# check if the result has the expected index
try:
    # assert type(date_sorter().index) == pd.RangeIndex
    # assert (date_sorter().index == pd.RangeIndex(start=0, stop=500, step=1)).all()
    assert list(date_sorter().index) == list(range(500))
    print("Passed index check")
except:
    print("Failed index check")
# check the tie-break sort for a sample of records where some have the same date
# note that this only tests a sample and does not check the entire answer
try:
    test_indices = [335, 415, 323, 405, 370, 382, 303, 488, 283,
                    395, 318, 369, 493, 252, 314, 410, 490]
    answer_lkp = {original_index:answer_index for
                  answer_index, original_index in s_test.to_dict().items()}
    i_test = [answer_lkp[i] for i in test_indices]
    assert sorted(i_test) == i_test
    print("Passed secondary sort sample check")
except:
    print("Failed secondary sort sample check")
    


True
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  Tr

In [29]:
import numpy as np
def run_v_check(s_test):
    print(s_test.iloc[10*0:(0+1)*10].values)
    print(np.array(range(1, 11)))
    print(s_test.iloc[10*0:(0+1)*10].values * np.array(range(1, 11)))
    print(s_test.iloc[10*0:(0+1)*10].values * np.array(range(1, 11)).sum())
    
    learner_checksums = []
    for i in range(50):
        block = s_test.iloc[10*i:(i+1)*10].values
        weights = np.array(range(1, 11))
        weighted_block = block * weights
        checksum = weighted_block.sum()
    
        # Print debugging info
        print(f"\nBlock {i}:")
        print(f"  Values: {block}")
        print(f"  Weights: {weights}")
        print(f"  Weighted values: {weighted_block}")
        print(f"  Checksum: {checksum}")
    
        #learner_checksums.append(checksum)
    
    '''print(s_test.iloc[10*i:(i+1)*10].values * np.array(range(1, 11))).sum()
                for i in range(50))'''
    """
    Check if the parsed dates appear to be correct and correctly sorted.
    The check works by producing some test checksums.
    The results of the test are printed and saved to a CSV.
    """
    try:
        
        v_check = pd.DataFrame({
            'correct': [
                6695, 14428, 16742, 9275, 12290, 14654, 9421, 10185, 11464, 16491,
                11797, 14036, 15459, 9412, 13069, 10400, 10498, 14322, 13274, 11001,
                11383, 11910, 10977, 9692, 10199, 10187, 15456, 13491, 9186, 13646,
                11142, 13724, 10994, 12905, 15968, 16648, 13966, 14607, 16932, 14622,
                17942, 18220, 17818, 18305, 19633, 12522, 13978, 18445, 20156, 14797
            ],
            'learner': [
                (s_test.iloc[10*i:(i+1)*10].values * np.array(range(1, 11))).sum()
                for i in range(50)
            ]
        }, index=range(0, 500, 10))

        v_check['agree'] = v_check['correct'] == v_check['learner']

        print("Values checksums:")
        print(v_check)

        # Save results to CSV for inspection
        v_check.to_csv('value_check_results.csv', index=True)

        # Print mismatches only
        mismatches = v_check[~v_check['agree']]
        if not mismatches.empty:
            print("\nMismatches:")
            print(mismatches)
        else:
            print("Passed values check ✅")

    except Exception as e:
        print("Failed values check ❌")
        print("Error:", e)

    return
run_v_check(s_test)

[  9  84   2  53  28 474 153  13 129  98]
[ 1  2  3  4  5  6  7  8  9 10]
[   9  168    6  212  140 2844 1071  104 1161  980]
[  495  4620   110  2915  1540 26070  8415   715  7095  5390]

Block 0:
  Values: [  9  84   2  53  28 474 153  13 129  98]
  Weights: [ 1  2  3  4  5  6  7  8  9 10]
  Weighted values: [   9  168    6  212  140 2844 1071  104 1161  980]
  Checksum: 6695

Block 1:
  Values: [111 225  31 171 191 486 335 415  36 323]
  Weights: [ 1  2  3  4  5  6  7  8  9 10]
  Weighted values: [ 111  450   93  684  955 2916 2345 3320  324 3230]
  Checksum: 14428

Block 2:
  Values: [405 422 375 380 345  57 481 436 104 299]
  Weights: [ 1  2  3  4  5  6  7  8  9 10]
  Weighted values: [ 405  844 1125 1520 1725  342 3367 3488  936 2990]
  Checksum: 16742

Block 3:
  Values: [162 154 402  95  73 108 156 332 182  82]
  Weights: [ 1  2  3  4  5  6  7  8  9 10]
  Weighted values: [ 162  308 1206  380  365  648 1092 2656 1638  820]
  Checksum: 9275

Block 4:
  Values: [351 278 214 155 2